<a href="https://colab.research.google.com/github/ElenaPribylova/-10/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%B5%D0%B9%D1%81_%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B5_%D1%81%D0%B5%D1%81%D1%81%D0%B8%D0%B8%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **3 кейс**

**Пользовательские сессии**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [9]:
!wget https://gist.github.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv

--2025-12-06 16:21:03--  https://gist.github.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv [following]
--2025-12-06 16:21:03--  https://gist.githubusercontent.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880014 (1.8M) [text/plain]
Saving to: ‘sessions.csv.2’

sessions.csv.2      100%[===================>]   1.79M  --.-KB/s    in 0.08s   

2025-12-06 16:21:03 (22.2 MB/s) - ‘session

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [17]:
import pandas as pd

df = pd.read_csv('sessions.csv')
df

,user_id,session
0,user0,"[""2023-05-06 20:43:45"", 1192, [[""page0"", ""2023..."
1,user1,"[""2023-05-03 15:39:47"", 1642, [[""page0"", ""2023..."
2,user1,"[""2023-05-16 21:24:30"", 506, [[""page0"", ""2023-..."
3,user1,"[""2023-05-09 01:41:35"", 1004, [[""page0"", ""2023..."
4,user1,"[""2023-05-26 05:31:50"", 984, [[""page0"", ""2023-..."
...,...,...
5278,user998,"[""2023-05-26 17:33:49"", 631, [[""page0"", ""2023-..."
5279,user998,"[""2023-05-15 23:21:39"", 1393, [[""page0"", ""2023..."
5280,user998,"[""2023-05-16 10:15:48"", 1448, [[""page0"", ""2023..."
5281,user998,"[""2023-05-26 17:06:25"", 810, [[""page0"", ""2023-..."


### **Решения**

#### **Задача 1**

Ваша задача написать функцию `get_time_to_conversion`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает среднее время между заходом на сайт и целевым действием.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res`

**Решение**

Напишите свое решение ниже

In [20]:
import csv
import json
import datetime

# Определяем функцию для десериализации объектов типа datetime
def deserialize_datetime(dct):
    for key, value in dct.items():
        if isinstance(value, str):
            try:
                dct[key] = datetime.datetime.fromisoformat(value)
            except ValueError:
                pass
    return dct

# Создаем пустой словарь
users_data = {}

# Открываем файл для чтения
with open('sessions.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)

    # Пропускаем заголовок
    next(reader)

    # Обходим каждую строку в файле
    for row in reader:
        user_id = row[0]
        session_json = row[1]

        # Десериализуем сессию из формата JSON
        session = json.loads(session_json, object_hook=deserialize_datetime)

        # Преобразуем дату и время начала сессии к типу datetime.datetime
        start_time = datetime.datetime.fromisoformat(session[0])

        # Преобразуем даты и времена посещения страниц к типу datetime.datetime и преобразуем список в кортеж
        pages = [(page[0], datetime.datetime.fromisoformat(page[1])) for page in session[2]]

        # Сохраняем данные в нужном формате
        users_data.setdefault(user_id, []).append((start_time, session[1], pages))


In [21]:
# Сколько времени обычно проходит между заходом на сайт и оставлением заявки
import statistics

def get_time_to_conversion(users_data):
    time_to_conversion = []

    for user_id in users_data:
        sessions = users_data[user_id]
        for session_start, session_duration, session_pages in sessions:
            conversion_page = "conversion"
            for page_name, page_view_time in session_pages:
                if page_name == conversion_page:
                    time_to_conversion.append((user_id, (page_view_time - session_start).seconds))
                    break

    values = [value for _, value in time_to_conversion]

    return round(statistics.mean(values), 2)

res = get_time_to_conversion(users_data)

In [22]:
try:
    assert res == 467.24
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `get_session_duration`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает сколько вообще в среднем длится сессия.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res2`

**Решение**

Напишите свое решение ниже

In [26]:
import statistics

def get_session_duration(users_data):

    if not users_data:
        return 0.0

    session_durations = []

    for user_sessions in users_data.values():
        for session_start, session_duration, session_pages in user_sessions:
            session_durations.append(session_duration)

    # Если сессий нет, возвращаем 0
    if not session_durations:
        return 0.0

    return round(statistics.mean(session_durations), 2)

res2 = get_session_duration(users_data)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [24]:
try:
    assert res2 == 901.35
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Напишите функцию `get_daily_session_count`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает cколько разных заходов в течение дня делает пользователь.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res3`

**Решение**

Напишите свое решение ниже

In [29]:

def get_daily_session_count(users_data):
    daily_session_count = []

    for user_id in users_data:
        sessions = users_data[user_id]
        session_days = {}

        for session_start, session_duration, session_pages in sessions:
            session_day = session_start.strftime("%Y-%m-%d")
            if session_day not in session_days:
                session_days[session_day] = 0
            session_days[session_day] += 1

        for session_day in session_days:
            daily_session_count.append((user_id, session_day, session_days[session_day]))

    values = [value for user, _, value in daily_session_count]

    return round(statistics.mean(values), 2)

res3 = get_daily_session_count(users_data)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [30]:
try:
    assert res3 == 1.11
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 4**

Напишите функцию `get_average_session_count`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает сколько заходов вообще в среднем делает пользователь.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res4`

**Решение**

Напишите свое решение ниже

In [31]:
def get_average_session_count(users_data):
    if not users_data:  # Проверка на пустой словарь
        return 0.0

    total_sessions = sum(len(sessions) for sessions in users_data.values())
    average = total_sessions / len(users_data)

    return round(average, 2)

res4 = get_average_session_count(users_data)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [32]:
try:
    assert res4 == 5.28
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
